# Determining Astrometric Acceleration for Each Star in the Sample

ACKNOWLEDGEMENT: Code based on accelerations code written by Elena Mitra.

This code uses proper motion and position values to determine the acceleration of a given system using the following equation: 

$$a_{\alpha\delta} = \frac{2d(\mu_{gaia} - \mu_{hg})}{\phi_{gaia} - \phi_{hip}}$$

The error propagation is derived using the standard error propagation equation. Starting with each acceleration individually, using the following equation (only RA shown, same for Dec): 

$$\sigma{a_{\alpha\delta(\alpha)}} = \frac{2}{\Delta\phi} \sqrt{d^2(\sigma\mu_{\alpha G}^2 + \sigma\mu_{\alpha HG}^2) + (\mu_{\alpha G} - \mu_{\alpha HG})^2\sigma d^2}$$

These two errors are then combined using the following equation for the total error on the astrometric acceleration:

$$\sigma a_{\alpha\delta} = \Bigg(\frac{a_{\alpha\delta RA}}{a_{\alpha\delta}}^2\bigg)\sigma a_{\alpha\delta RA}^2 + \bigg(\frac{a_{\alpha\delta Dec}}{a_{\alpha\delta}}\bigg)\sigma a_{\alpha\delta Dec}^2 \pm \frac{2a_{\alpha\delta RA}a_{\alpha\delta Dec}}{a_{\alpha\delta}^2}\sigma a_{\alpha\delta RA}a_{\alpha\delta Dec}$$

The variables are defined as follows: 

- $a_{\alpha\delta}$ = astrometric acceleration in $m/s/yr$
- $d$ = distance in $pc$
- $\mu_{gaia}$ = proper motion from Gaia in $mas/yr$
- $\mu_{hg}$ = proper motion in HGCA for the combined Hipparcos-Gaia measurement in $mas/yr$
- $\phi_{gaia}$ = epoch of proper motion measurement from Gaia in $years$
- $\phi_{hip}$ = epoch of proper motion measurement from Hipparcos in $years$

In [24]:
# Necessary modules 
import pandas as pd
import astropy.units as u
import numpy as np
import os

# Delete previous electronic table so that it can be overwritten 
os.remove('/Users/Jess/HGCA_survey_paper/accelerations_electronic_table.csv')

The first step is to pull all the data we will need to calculate the accelerations. This includes the proper motions and parallax values. User should start by loading these dataframes and creating variables for each of the necessary columns. These are:

- Parallax and associated error 
- HIP ID or star ID
- RA proper motion for each star from Hipparcos and associated error
- Dec proper motion for each star from Hipparcos and associated error
- RA proper motion for each star from Gaia and associated error 
- Dec proper motion for each star from Gaia and associated error
- RA proper motion for each star as a combination of Hipparcos and Gaia as determined by the HGCA catalog and associated error
- Dec proper motion for each star as a combination of Hipparcos and Gaia as determined by the HGCA catalog and associated error
- Epoch values for the Hipparcos measurement and associated error 
- Epoch values for the Gaia measurement and associated error


In [25]:
# Pulling the sample to search.
my_sample = pd.read_csv('/Users/Jess/HGCA_survey_paper/completed_cross_match.csv')
pmdec_hip_file = pd.read_csv('/Users/Jess/HGCA_survey_paper/pmdec_hip.csv')

print('The cross-match file has ' + str(len(my_sample)) + ' entries.')
print('The pmdec metric list has ' + str(len(pmdec_hip_file)) + ' entries.\n')
print('The first five items in the my_sample file are: ' + str(my_sample[0:5]) + '.')
print('The first five items in the pmdec_hip file are: ' + str(pmdec_hip_file[0:5]) + '.')

# Pulling necessary values from the data table.
print('The length and first five values of each of these lists are as follows:\n')

parallax = my_sample['gaia_parallax']
print(str(len(parallax)) + ' parallax measurements. First five: ' + str(list(parallax[0:5])) + '.\n')

err_parallax = my_sample['gaia_parallex_err']
print(str(len(err_parallax)) + ' parallax error measurements. First five: ' + str(list(err_parallax[0:5])) + '.\n')

hip_id = my_sample['hip_id']
print(str(len(hip_id)) + ' HIP IDs. First five: ' + str(list(hip_id[0:5])) + '.\n')

pmra_hip = my_sample['pmra_hip']
print(str(len(pmra_hip)) + ' Hipparcos proper motion in RA measurements. First five: ' + str(list(pmra_hip[0:5])) + '.\n')

err_pmra_hip = my_sample['pmra_hip_error']
print(str(len(err_pmra_hip)) + ' Hipparcos proper motion in RA error measurements. First five: ' + str(list(err_pmra_hip[0:5])) + '.\n')

pmdec_hip = pmdec_hip_file['pmdec_hip']
print(str(len(pmdec_hip)) + ' Hipparcos proper motion in Dec measurements. First five: ' + str(list(pmdec_hip[0:5])) + '.\n')

err_pmdec_hip = my_sample['pmdec_hip_error']
print(str(len(err_pmdec_hip)) + ' Hipparcos proper motion in Dec error measurements. First five: ' + str(list(err_pmdec_hip[0:5])) + '.\n')

pmra_gaia = my_sample['pmra_gaia']
print(str(len(pmra_gaia)) + ' Gaia proper motion in RA measurements. First five: ' + str(list(pmra_gaia[0:5])) + '.\n')

err_pmra_gaia = my_sample['pmra_gaia_err']
print(str(len(err_pmra_gaia)) + ' Gaia proper motion in RA error measurements. First five: ' + str(list(err_pmra_gaia[0:5])) + '.\n')

pmdec_gaia = my_sample['pmdec_gaia']
print(str(len(pmdec_gaia)) + ' Gaia proper motion in Dec measurements. First five: ' + str(list(pmdec_gaia[0:5])) + '.\n')

err_pmdec_gaia = my_sample['pmdec_gaia_err']
print(str(len(err_pmdec_gaia)) + ' Gaia proper motion in Dec error measurements. First five: ' + str(list(err_pmdec_gaia[0:5])) + '.\n')

pmra_hg = my_sample['pmra_hg']
print(str(len(pmra_hg)) + ' combined proper motion in RA measurements. First five: ' + str(list(pmra_hg[0:5])) + '.\n')

err_pmra_hg = my_sample['pmra_hg_error']
print(str(len(err_pmra_hg)) + ' combined proper motion in RA error measurements. First five: ' + str(list(err_pmra_hg[0:5])) + '.\n')

pmdec_hg = my_sample['pmdec_hg']
print(str(len(pmdec_hg)) + ' combined proper motion in Dec measurements. First five: ' + str(list(pmdec_hg[0:5])) + '.\n')

err_pmdec_hg = my_sample['pmdec_hg_error']
print(str(len(err_pmdec_hg)) + ' combined proper motion in Dec error measurements. First five: ' + str(list(err_pmdec_hg[0:5])) + '.\n')

epoch_ra_gaia = my_sample['epoch_ra_gaia']
print(str(len(epoch_ra_gaia)) + ' Gaia epoch in RA measurements. First five: ' + str(list(epoch_ra_gaia[0:5])) + '.\n')

epoch_dec_gaia = my_sample['epoch_dec_gaia']
print(str(len(epoch_dec_gaia)) + ' Gaia epoch in Dec measurements. First five: ' + str(list(epoch_dec_gaia[0:5])) + '.\n')

epoch_ra_hip = my_sample['epoch_ra_hip']
print(str(len(epoch_ra_hip)) + ' Hipparcos epoch in RA measurements. First five: ' + str(list(epoch_ra_hip[0:5])) + '.\n')

epoch_dec_hip = my_sample['epoch_dec_hip']
print(str(len(epoch_dec_hip)) + ' Hipparcos epoch in Dec measurements. First five: ' + str(list(epoch_dec_hip[0:5])) + '.\n')

The cross-match file has 1386 entries.
The pmdec metric list has 1386 entries.

The first five items in the my_sample file are:    hip_id              gaia_id          ra  ra_error        dec  dec_error  \
0    8387    91429481386429184   27.045801  0.131704  16.955387   0.105072   
1   58684  1537396673318780544  180.526415  0.205209  43.045939   0.159836   
2   14669   434690036554889856   47.375779  0.715960  45.731367   0.744419   
3   39816   669664811098938240  122.035912  0.036658  19.215539   0.023396   
4   50303   741357981454593536  154.059776  0.172669  29.310376   0.103583   

     rad_vel  rad_vel_err  gaia_parallax  gaia_parallex_err  ...  lee_var  \
0        NaN          NaN      11.461327           0.151113  ...      0.0   
1        NaN          NaN      28.203178           0.225391  ...      3.0   
2  -4.555012     0.912451      57.108803           0.819751  ...      0.0   
3  26.918345     0.170162      12.779728           0.039784  ...      0.0   
4        NaN      

There are a lot of units and conversions to keep track of in this calculation so at this point, each of the variables are saved into lists and assigned a quantity using astropy quantity, as follows: 

- $mas/yr$ for each of the proper motion values and associated errors
- $mas$ for the parallax values and associated errors
- $year$ for the epoch values and associated errors

In [26]:
pmrahipl, pmrahipel, pmragaial, pmragaiael, pmrahgl, pmrahgel, pmdechipl, pmdechipel, pmdecgaial, pmdecgaiael, pmdechgl, pmdechgel, rahipepochl, ragaiaepochl, dechipepochl, decgaiaepochl, plxl, plxel = ([] for i in range(18))

# Cycle through each star and create lists of the values needed in the right units. 
for i in range(len(hip_id)):

    pmrahipl.append(pmra_hip.iloc[i]*u.mas/u.yr) #mas/yr
    pmrahipel.append(err_pmra_hip.iloc[i]*u.mas/u.yr)
    
    pmragaial.append(pmra_gaia.iloc[i]*u.mas/u.yr) #mas/yr
    pmragaiael.append(err_pmra_gaia.iloc[i]*u.mas/u.yr)
    
    pmrahgl.append(pmra_hg.iloc[i]*u.mas/u.yr) #mas/yr
    pmrahgel.append(err_pmra_hg.iloc[i]*u.mas/u.yr)
    
    pmdechipl.append(pmdec_hip.iloc[i]*u.mas/u.yr) #mas/yr
    pmdechipel.append(err_pmdec_hip.iloc[i]*u.mas/u.yr)
    
    pmdecgaial.append(pmdec_gaia.iloc[i]*u.mas/u.yr) #mas/yr
    pmdecgaiael.append(err_pmdec_gaia.iloc[i]*u.mas/u.yr)
    
    pmdechgl.append(pmdec_hg.iloc[i]*u.mas/u.yr) #mas/yr
    pmdechgel.append(err_pmdec_hg.iloc[i]*u.mas/u.yr)
    
    rahipepochl.append(epoch_ra_hip.iloc[i]*u.yr) #yr
    ragaiaepochl.append(epoch_ra_gaia.iloc[i]*u.yr) #yr
    
    dechipepochl.append(epoch_dec_hip.iloc[i]*u.yr) #yr
    decgaiaepochl.append(epoch_dec_gaia.iloc[i]*u.yr) #yr
    
    plxl.append((parallax.iloc[i])*u.mas) #parallax in mas
    plxel.append((err_parallax.iloc[i])*u.mas)
    
# Checks:
print('The proper motion in Hipparcos RA will be in units of mas/yr ' + str(pmrahipl[0:5]) + '.\n')
print('The error in the Hipparcos proper motion in RA will be in units of mas/yr ' + str(pmrahipel[0:5]) + '.\n')
print('The proper motion in Gaia RA will be in units of mas/yr ' + str(pmragaial[0:5]) + '.\n')
print('The error in the Gaia proper motion in RA will be in units of mas/yr ' + str(pmragaiael[0:5]) + '.\n')
print('The combined proper motion in RA will be in units of mas/yr ' + str(pmrahgl[0:5]) + '.\n')
print('The error in the combined proper motion in RA will be in units of mas/yr ' + str(pmrahgel[0:5]) + '.\n')
print('The proper motion in Hipparcos Dec will be in units of mas/yr ' + str(pmdechipl[0:5]) + '.\n')
print('The error in the Hipparcos proper moton in Dec will be in units of mas/yr ' + str(pmdechipel[0:5]) + '.\n')
print('The proper motion in Gaia Dec will be in units of mas/yr ' + str(pmdecgaial[0:5]) + '.\n')
print('The error in the Gaia proper motion in Dec will be in units of mas/yr ' + str(pmdecgaiael[0:5]) + '.\n')
print('The combined proper motion in Dec will be in units of mas/yr ' + str(pmdechgl[0:5]) + '.\n')
print('The error in the combined proper motion in Dec will be in units of mas/yr ' + str(pmdechgel[0:5]) + '.\n')
print('The Hipparcos RA epoch is in units of yr ' + str(rahipepochl[0:5]) + '.\n')
print('The Gaia RA epoch is in units of yr ' + str(ragaiaepochl[0:5]) + '.\n')
print('The Hipparcos Dec epoch is in units of yr ' + str(dechipepochl[0:5]) + '.\n')
print('The Gaia Dec epoch is in units of yr ' + str(decgaiaepochl[0:5]) + '.\n')
print('The parallax will have units of mas ' + str(plxl[0:5]) + '.\n')
print('The error in parallax will have units of mas ' + str(plxel[0:5]) + '.\n')

The proper motion in Hipparcos RA will be in units of mas/yr [<Quantity 55.18491745 mas / yr>, <Quantity -347.33679199 mas / yr>, <Quantity -429.67767334 mas / yr>, <Quantity -35.62554169 mas / yr>, <Quantity -71.22349548 mas / yr>].

The error in the Hipparcos proper motion in RA will be in units of mas/yr [<Quantity 0.60710633 mas / yr>, <Quantity 0.7495575 mas / yr>, <Quantity 3.01097035 mas / yr>, <Quantity 0.85253072 mas / yr>, <Quantity 0.48353976 mas / yr>].

The proper motion in Gaia RA will be in units of mas/yr [<Quantity 65.60846491 mas / yr>, <Quantity -316.32185305 mas / yr>, <Quantity -422.17907314 mas / yr>, <Quantity -35.71537608 mas / yr>, <Quantity -70.58981126 mas / yr>].

The error in the Gaia proper motion in RA will be in units of mas/yr [<Quantity 0.39525598 mas / yr>, <Quantity 0.58281732 mas / yr>, <Quantity 2.99749494 mas / yr>, <Quantity 0.1237405 mas / yr>, <Quantity 0.41452056 mas / yr>].

The combined proper motion in RA will be in units of mas/yr [<Quanti

The distance is needed in the equation of acceleration and can be computed from the Gaia parallax. The quantity of parsec is applied to both the distance and the error. Equations used: 

$$d = \frac{1}{\varpi}$$ 

$$\delta_{d} = \frac{\delta\varpi}{\varpi^2} $$

where $\varpi$ = parallax.

In [27]:
# Determine the distance to each of the stars.
d_list = []
d_err_list = []

for i in range(len(plxl)):    
    d = plxl[i].to(u.parsec, equivalencies=u.parallax())
    d_list.append(d)
    
    plxl_quant = plxl[i].to(u.parsec, equivalencies=u.parallax())
    plxl_err_quant = plxel[i].to(u.parsec, equivalencies=u.parallax())
    
    d = ((plxl_quant)**2) / (plxl_err_quant)
    d_err_list.append(d.to(u.pc))
    
print('The length of the distance list in quality is ' + str(len(d_list)) + '.')
print('The length of the distance error list is ' + str(len(d_err_list)) + '.\n')

print('The first five values of distance in quantity are ' + str(d_list[0:5]) + '.')
print('The first give values of distance error are ' + str(d_err_list[0:5]) + '.\n')

The length of the distance list in quality is 1386.
The length of the distance error list is 1386.

The first five values of distance in quantity are [<Quantity 87.24993493 pc>, <Quantity 35.45699657 pc>, <Quantity 17.51043536 pc>, <Quantity 78.24892713 pc>, <Quantity 85.52823581 pc>].
The first give values of distance error are [<Quantity 1.15035876 pc>, <Quantity 0.28336075 pc>, <Quantity 0.25134824 pc>, <Quantity 0.24359476 pc>, <Quantity 1.06012268 pc>].



Now the acceleration for RA and Dec can be calcuated. The numerator and denominator are calculated seperately so that the units can be converted for the final acceleration equation. The units of the numerator are $m/s$ and the units of the denominator are $years$.

In [28]:
# Set up lists for acceperations that will be calculated. 
acc_ra_numl, acc_ra_denl, acc_dec_numl, acc_dec_denl, acc_ra_num2, acc_dec_num2 = ([] for i in range(6))

# Cycle through all stars and determine the numerator and denominator of later equations. 
# del mu (in mas/yr) * parallax (in parsec) converted to (mas pc/yr)
for i in range(len(d_list)):
#    # Acceleration in RA
    acc_ra_numl.append((pmragaial[i] - pmrahgl[i]) * d_list[i]) # Numerator 
    acc_ra_denl.append((ragaiaepochl[i] - rahipepochl[i]) / 2) # Denominator
#    
#    # Acceleration in Dec
    acc_dec_numl.append((pmdecgaial[i] - pmdechgl[i]) * d_list[i]) # Numerator 
    acc_dec_denl.append((decgaiaepochl[i] - dechipepochl[i]) / 2) # Demoninator
#    
#    # Change the units in both of the numerators
    acc_ra_num2.append(acc_ra_numl[i].to(u.m / u.second, equivalencies=u.dimensionless_angles()))
    acc_dec_num2.append(acc_dec_numl[i].to(u.m / u.second, equivalencies=u.dimensionless_angles()))
    
print('The first five values of numerator RA acceleration are ' + str(acc_ra_numl[0:5]) + '.\n')
print('The first five values of denominator RA acceleration are ' + str(acc_ra_denl[0:5]) + '.\n')
print('The first five values of numerator Dec acceleration are ' + str(acc_dec_numl[0:5]) + '.\n')
print('The first five values of denominator Dec acceleration are ' + str(acc_dec_denl[0:5]) + '.\n')

The first five values of numerator RA acceleration are [<Quantity 1097.22039444 mas pc / yr>, <Quantity 374.78993181 mas pc / yr>, <Quantity -98.04832156 mas pc / yr>, <Quantity -16.38341305 mas pc / yr>, <Quantity 104.6324862 mas pc / yr>].

The first five values of denominator RA acceleration are [<Quantity 12.14281453 yr>, <Quantity 12.22654211 yr>, <Quantity 12.17391643 yr>, <Quantity 12.03299839 yr>, <Quantity 12.13182316 yr>].

The first five values of numerator Dec acceleration are [<Quantity 599.16600531 mas pc / yr>, <Quantity 544.98674365 mas pc / yr>, <Quantity -663.89470976 mas pc / yr>, <Quantity 23.31940987 mas pc / yr>, <Quantity 379.5267378 mas pc / yr>].

The first five values of denominator Dec acceleration are [<Quantity 12.08015967 yr>, <Quantity 11.92207928 yr>, <Quantity 12.16522058 yr>, <Quantity 12.11862158 yr>, <Quantity 12.22736358 yr>].



The numerator and denominator are now combined to create an acceleration value in RA and Dec for each star. The final value unit is $m/s/yr$. 

In [29]:
# Use these numerators and denominators to determine the acceleration in RA and Dec
acc_ral, acc_rael, acc_decl, acc_decel =  [], [], [], []

for i in range(len(acc_ra_numl)):
    # Acceleration in RA and Dec
    acc_ral.append(acc_ra_num2[i] / acc_ra_denl[i]) # RA 
    acc_decl.append(acc_dec_num2[i] / acc_dec_denl[i]) # Dec)
    
print('There are ' + str(len(acc_ral)) + ' RA acceleration values.')
print('There are ' + str(len(acc_decl)) + ' Dec acceleration values.\n')

print('The first five RA values of acceleration are ' + str(acc_ral[0:5]) + '.')
print('The first five Dec values of acceleration are ' + str(acc_decl[0:5]) + '.')

There are 1386 RA acceleration values.
There are 1386 Dec acceleration values.

The first five RA values of acceleration are [<Quantity 428.34722191 m / (s yr)>, <Quantity 145.31341612 m / (s yr)>, <Quantity -38.17959281 m / (s yr)>, <Quantity -6.4543419 m / (s yr)>, <Quantity 40.88480387 m / (s yr)>].
The first five Dec values of acceleration are [<Quantity 235.12344444 m / (s yr)>, <Quantity 216.69823696 m / (s yr)>, <Quantity -258.70252336 m / (s yr)>, <Quantity 9.12190987 m / (s yr)>, <Quantity 147.14008293 m / (s yr)>].


The error for the RA and Dec accelerations are now propagated. 

In [30]:
ra_error = []
dec_error = []

for i in range(len(pmra_gaia)):
    const = 2 / (ragaiaepochl[i] - rahipepochl[i])
    
    term_1 = (d_list[i] * pmragaiael[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    term_2 = (d_list[i] * pmrahgel[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    term_3 = ((pmragaial[i] - pmrahgl[i]) * d_err_list[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    
    ra_err = const * np.sqrt((term_1)**2 + (term_2)**2 + (term_3)**2)
    ra_error.append(ra_err)
    
    const_2 = 2 / (decgaiaepochl[i] - dechipepochl[i])
    
    term_a = (d_list[i] * pmdecgaiael[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    term_b = (d_list[i] * pmdechgel[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    term_c = ((pmdecgaial[i] - pmdechgl[i]) * d_err_list[i]).to(u.m / u.second, equivalencies=u.dimensionless_angles())
    
    dec_err = const_2 * np.sqrt((term_a)**2 + (term_b)**2 + (term_c)**2)
    dec_error.append(dec_err)
    
print('There are ' + str(len(ra_error)) + ' errors for RA.')
print('There are ' + str(len(dec_error)) + ' errors for Dec.\n')

print('The first five values for RA error are: ' + str(ra_error[0:5]) + '.')
print('The first five values for Dec error are: ' + str(dec_error[0:5]) + '.')

There are 1386 errors for RA.
There are 1386 errors for Dec.

The first five values for RA error are: [<Quantity 14.62049869 m / (s yr)>, <Quantity 8.10392877 m / (s yr)>, <Quantity 20.47974946 m / (s yr)>, <Quantity 3.94768982 m / (s yr)>, <Quantity 13.88067309 m / (s yr)>].
The first five values for Dec error are: [<Quantity 7.84619308 m / (s yr)>, <Quantity 4.71364005 m / (s yr)>, <Quantity 12.59718805 m / (s yr)>, <Quantity 1.6658692 m / (s yr)>, <Quantity 6.99186882 m / (s yr)>].


The RA and Dec accelerations are then combined to find an overall acceleration for the system with a unit of $m/s/yr$. 

In [31]:
# Combine the acceleration in RA and Dec in quadrature to determine the astrometric acceleration, and save to a list.
astrometric_accel_quantity = []

for i in range(len(acc_ral)):
    astrometric_acc = np.sqrt(acc_ral[i] ** 2 + acc_decl[i] ** 2)
    astrometric_accel_quantity.append(astrometric_acc)

print('There are ' + str(len(astrometric_accel_quantity)) + ' values of acceleration.\n')

print('The first five values of astrometric acceleration are: ' + str(astrometric_accel_quantity[0:5]))

There are 1386 values of acceleration.

The first five values of astrometric acceleration are: [<Quantity 488.63521838 m / (s yr)>, <Quantity 260.91016616 m / (s yr)>, <Quantity 261.5046403 m / (s yr)>, <Quantity 11.17442477 m / (s yr)>, <Quantity 152.71467248 m / (s yr)>]


The error for the total astrometric acceleration can then be propagated. 

In [32]:
accel_error = []

for i in range(len(ra_error)):
    term_1 = (acc_ral[i] / astrometric_accel_quantity[i])**2 * (ra_error[i])**2 
    
    term_2 = (acc_decl[i] / astrometric_accel_quantity[i])**2 * (dec_error[i])**2 
    covariance = np.cov([ra_error[i].value, dec_error[i].value])
    covariance_value = covariance * (u.m / u.second / u.year)**2
    
    term_3 = (2 * acc_ral[i] * acc_decl[i] * covariance_value) / astrometric_accel_quantity[i]**2
    
    error = np.sqrt(term_1 + term_2 + term_3)
    
    accel_error.append(error)
    
print(accel_error[0:5])

[<Quantity 14.06688962 m / (s yr)>, <Quantity 6.40426936 m / (s yr)>, <Quantity 13.16135688 m / (s yr)>, <Quantity 2.14325098 m / (s yr)>, <Quantity 8.45180962 m / (s yr)>]


For each of the six variables calcuated, they are converted back into floats from quantities so that they can be saved into an easy to manipulate CSV file. The following is a list of calculated terms that are saved:

- Astrometric acceleration in RA and associated error
- Astrometric acceleration in Dec and associated error
- Astrometric acceleration combined and associated error

In [33]:
astrometric_accel = []
ra_acceleration = []
dec_acceleration = []
ra_acceleration_error = []
dec_acceleration_error = []
full_acceleration_error = []

for i in range(len(astrometric_accel_quantity)):
    str_i = str(astrometric_accel_quantity[i])
    str_i_cut = str_i[0:-11]
    astrometric_accel.append(float(str_i_cut))
    
    str_j = str(acc_ral[i])
    str_j_cut = str_j[0:-11]
    ra_acceleration.append(float(str_j_cut))
    
    str_k = str(acc_decl[i])
    str_k_cut = str_k[0:-11]
    dec_acceleration.append(float(str_k_cut))
    
    str_l = str(ra_error[i])
    str_l_cut = str_l[0:-11]
    ra_acceleration_error.append(float(str_l_cut))
    
    str_m = str(dec_error[i])
    str_m_cut = str_m[0:-11]
    dec_acceleration_error.append(float(str_m_cut))
    
    str_n = str(accel_error[i])
    str_n_cut = str_n[0:-11]
    full_acceleration_error.append(float(str_n_cut))
    
    
print('There are ' + str(len(astrometric_accel)) + ' astrometric acceleration values.')
print('There are ' + str(len(ra_acceleration)) + ' astrometric acceleration in RA values.')
print('There are ' + str(len(dec_acceleration)) + ' astrometric acceleration in Dec values.')
print('There are ' + str(len(ra_acceleration_error)) + ' astrometric acceleration in RA error values.')
print('There are ' + str(len(dec_acceleration_error)) + ' astrometric acceleration in Dec error values')
print('There are ' + str(len(full_acceleration_error)) + ' astrometric acceleration error values.\n')

print('The first five values of the astrometric acceleration are ' + str(astrometric_accel[0:5]))
print('The first five values of the RA acceleration are ' + str(ra_acceleration[0:5]))
print('The first five values of the Dec acceleration are ' + str(dec_acceleration[0:5]))
print('The first five values of the RA acceleration error are ' + str(ra_acceleration_error[0:5]))
print('The first five values of the Dec acceleration error are ' + str(dec_acceleration_error[0:5]))
print('The first five values of the acceleration error are ' + str(full_acceleration_error[0:5]))

There are 1386 astrometric acceleration values.
There are 1386 astrometric acceleration in RA values.
There are 1386 astrometric acceleration in Dec values.
There are 1386 astrometric acceleration in RA error values.
There are 1386 astrometric acceleration in Dec error values
There are 1386 astrometric acceleration error values.

The first five values of the astrometric acceleration are [488.6352183836913, 260.9101661578318, 261.5046403021705, 11.174424770070038, 152.71467248124998]
The first five values of the RA acceleration are [428.3472219092604, 145.31341611823484, -38.17959281450005, -6.454341895375549, 40.88480386756969]
The first five values of the Dec acceleration are [235.12344444460777, 216.69823695765527, -258.70252335856054, 9.121909867980213, 147.14008292700464]
The first five values of the RA acceleration error are [14.620498691285498, 8.103928774860465, 20.479749460876175, 3.947689819292848, 13.880673088518893]
The first five values of the Dec acceleration error are [7.

These are first saved into a CSV file that can be manipulated further in the work for this survey. They are then also saved into a machine readable table that can be distributed with the publication.  

In [34]:
# Create a csv file with the acceleration column
header = ['hip_id', 'acceleration', 'accel_err', 'accel_ra', 'accel_dec', 'ra_err', 'dec_err']
row = zip(hip_id, astrometric_accel, full_acceleration_error, ra_acceleration, dec_acceleration, ra_acceleration_error, dec_acceleration_error)
accel_file = pd.DataFrame(row, columns=header)
accel_file.to_csv('/Users/Jess/HGCA_survey_paper/accelerations.csv', index=False)

The following provides the output for the electronic table. A header should be attached, found in the txt file acceleration_table_header. 

In [35]:
header = ['hip_id', 'accel_ra', 'ra_err', 'accel_dec', 'dec_err', 'acceleration', 'accel_err']
row = zip(hip_id, ra_acceleration, ra_acceleration_error, dec_acceleration, dec_acceleration_error, astrometric_accel, full_acceleration_error)
accel_file = pd.DataFrame(row, columns=header)
accel_file.to_csv('/Users/Jess/HGCA_survey_paper/accelerations_electronic_table.csv', index=False)